In [3]:
import cv2
import numpy as np
import pandas as pd
import os
import glob
import json
import shutil
import sys
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt

def find_project_root(marker=".gitignore"):
    current = Path.cwd()
    for parent in [current] + list(current.parents):
        if (parent / marker).exists():
            return parent.resolve()
    raise FileNotFoundError(
        f"Project root marker '{marker}' not found starting from {current}")
    
root = find_project_root()
src_path = f"{root}/src"
if src_path not in sys.path:
    sys.path.append(src_path)

### Examples of anomalies and valid reprojections (with good calibration)

In [4]:
valid_example = f"{root}/reprojected_frames_B/frame_1103" # very accurate
valid_example_2 = f"{root}/reprojected_frames_B/frame_1155" # very accurate
valid_example_3 = f"{root}/reprojected_frames_B/frame_1171" # less accurate but valid

anomalie_example = f"{root}/reprojected_frames_B/frame_1354"
anomalie_example_2 = f"{root}/reprojected_frames_B/frame_1385"

### Example of valid calibration triangulation (good vs bad calibration)

In [ ]:
# search for a frame that was flagged as anomaly but was just bad triangulation
# then show the same frame flagged as valid due to good calibration